In [1]:
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
from IPython.display import display
import time
import re 
import datetime
import glob
import os

In [2]:
gu_list1 = ['종로구', '중구', '용산구', '성동구', '광진구',] 
gu_list2 = ['동대문구', '중랑구', '성북구', '강북구', '도봉구']
gu_list3 = ['노원구', '은평구', '서대문구', '마포구', '양천구'] 
gu_list4 = ['강서구', '구로구', '금천구', '영등포구', '동작구']
gu_list5 = ['관악구', '서초구', '강남구', '송파구', '강동구']

gu_list = gu_list1 + gu_list2 + gu_list3 + gu_list4 + gu_list5
gu_list

['종로구',
 '중구',
 '용산구',
 '성동구',
 '광진구',
 '동대문구',
 '중랑구',
 '성북구',
 '강북구',
 '도봉구',
 '노원구',
 '은평구',
 '서대문구',
 '마포구',
 '양천구',
 '강서구',
 '구로구',
 '금천구',
 '영등포구',
 '동작구',
 '관악구',
 '서초구',
 '강남구',
 '송파구',
 '강동구']

In [3]:
driver = webdriver.Chrome('./chromedriver')
driver.implicitly_wait(10)

gift = '선물세트'

url_dict = {}
for region_gu in gu_list: 
    driver.get(f'https://www.daangn.com/search/{region_gu}%20{gift}')

    for i in range(10): 
        driver.find_element_by_class_name("more-btn").click()
        time.sleep(1)

    # get > url data 
    hrefs = driver.find_elements_by_class_name('articles-wrap > article')

    url_list = []
    for href in hrefs:
        url = href.find_element_by_tag_name('a').get_attribute('href')
        url_list.append(url)
        
    url_list = url_list[:-7]
    
    url_dict[region_gu] = list(set(url_list))
    
    print(f"{region_gu} 주소 개수:", len(url_list))
    print(f"{region_gu} 중복제거 주소 개수:", len(list(set(url_list))))

    # export table > url data 
#     current = datetime.datetime.now().replace(microsecond=0).strftime("%Y-%m-%d %H%M%S")
#     url_df = pd.Series(list(set(url_list)), name='url').to_frame()
#     url_df.to_excel(f"carrot_url_{region_gu}_{gift}_{current}.xlsx")

종로구 주소 개수: 122
종로구 중복제거 주소 개수: 122
중구 주소 개수: 127
중구 중복제거 주소 개수: 126
용산구 주소 개수: 122
용산구 중복제거 주소 개수: 121
성동구 주소 개수: 125
성동구 중복제거 주소 개수: 123
광진구 주소 개수: 127
광진구 중복제거 주소 개수: 127
동대문구 주소 개수: 130
동대문구 중복제거 주소 개수: 130
중랑구 주소 개수: 125
중랑구 중복제거 주소 개수: 125
성북구 주소 개수: 125
성북구 중복제거 주소 개수: 125
강북구 주소 개수: 125
강북구 중복제거 주소 개수: 124
도봉구 주소 개수: 126
도봉구 중복제거 주소 개수: 125
노원구 주소 개수: 126
노원구 중복제거 주소 개수: 125
은평구 주소 개수: 127
은평구 중복제거 주소 개수: 127
서대문구 주소 개수: 125
서대문구 중복제거 주소 개수: 125
마포구 주소 개수: 126
마포구 중복제거 주소 개수: 123
양천구 주소 개수: 129
양천구 중복제거 주소 개수: 127
강서구 주소 개수: 130
강서구 중복제거 주소 개수: 124
구로구 주소 개수: 126
구로구 중복제거 주소 개수: 122
금천구 주소 개수: 126
금천구 중복제거 주소 개수: 126
영등포구 주소 개수: 127
영등포구 중복제거 주소 개수: 127
동작구 주소 개수: 127
동작구 중복제거 주소 개수: 126
관악구 주소 개수: 128
관악구 중복제거 주소 개수: 123
서초구 주소 개수: 129
서초구 중복제거 주소 개수: 119
강남구 주소 개수: 129
강남구 중복제거 주소 개수: 121
송파구 주소 개수: 126
송파구 중복제거 주소 개수: 118
강동구 주소 개수: 127
강동구 중복제거 주소 개수: 125


In [4]:
# get > carrot data 


for region_gu in list(url_dict.keys()): 

    carrot_list = []

    # start time 
    start_time = datetime.datetime.now().replace(microsecond=0)
    start_time_str = start_time.strftime("%Y-%m-%d %H%M%S")
    print(f"{region_gu} 시작 :", start_time_str)

    for url in url_dict[region_gu]:
        res = requests.get(url)
        soup = BeautifulSoup(res.content, "html.parser")
        
#         driver.get(url)
#         driver.implicitly_wait(10)
    #     time.sleep(1)

        try:
            num = int(url[-9:])
            region = soup.select('#region-name')[0].text

            title = soup.select('#article-title')[0].text

            categorys = soup.select('#article-category')[0].text.split("∙")
            category = categorys[0].strip()
            time1 = categorys[1].strip()

            try :
                price = soup.select('#article-price')[0].text.replace(",", "").strip()
                price = int(price.replace("원", ""))
            except:
                price = 0
            detail = soup.select('#article-detail > p')[0].text

            counts = soup.select("#article-counts")[0].text.split("∙")

            interest = int(counts[0].strip()[3:])
            chat = int(counts[1].strip()[3:])
            view = int(counts[2].strip()[3:])

            columns = [num, region, title, category, time1, price, detail, interest, chat, view]
            carrot_list.append(columns)

        except:
            pass

    

    # Dataframe
    columns = ['num', 'region', 'title', 'category', 'time', 'price', 'detail', 'interest', 'chat', 'view']
    df = pd.DataFrame(carrot_list, columns=columns) 
    print(f"{region_gu} 중복제거 전:", len(df))
    df = df.drop_duplicates()
    print(f"{region_gu} 중복제거 후:", len(df))

    # time
    finish_time = datetime.datetime.now().replace(microsecond=0)
    finish_time_str = finish_time.strftime("%Y-%m-%d %H%M%S")
    time_delta = finish_time - start_time
    print(f"{region_gu} 종료 :", finish_time_str)
    print(f"{region_gu} 소요시간 :", time_delta)
    print('-'*30)

    # export table > carrot data 
    file_name = f"carrot_list_{region_gu}_{gift}_{finish_time_str}.xlsx"
    df.to_excel(file_name, index=False)

    # export table > data list 
    data_list = pd.read_excel("data_list.xlsx")
    data_list = data_list.append({
        "file_name": file_name, 
        "start_time" : start_time,
        "finish_time" : finish_time, 
        "time_delta_seconds" : time_delta.total_seconds(), 
        "counts" : len(df)    
    }, ignore_index = True)

    data_list.to_excel("data_list.xlsx", index=False)
    
driver.quit()

종로구 시작 : 2021-10-08 102705
종로구 중복제거 전: 122
종로구 중복제거 후: 122
종로구 종료 : 2021-10-08 102741
종로구 소요시간 : 0:00:36
------------------------------
중구 시작 : 2021-10-08 102742
중구 중복제거 전: 125
중구 중복제거 후: 125
중구 종료 : 2021-10-08 102819
중구 소요시간 : 0:00:37
------------------------------
용산구 시작 : 2021-10-08 102820
용산구 중복제거 전: 121
용산구 중복제거 후: 121
용산구 종료 : 2021-10-08 102855
용산구 소요시간 : 0:00:35
------------------------------
성동구 시작 : 2021-10-08 102856
성동구 중복제거 전: 122
성동구 중복제거 후: 122
성동구 종료 : 2021-10-08 102932
성동구 소요시간 : 0:00:36
------------------------------
광진구 시작 : 2021-10-08 102932
광진구 중복제거 전: 126
광진구 중복제거 후: 126
광진구 종료 : 2021-10-08 103011
광진구 소요시간 : 0:00:39
------------------------------
동대문구 시작 : 2021-10-08 103011
동대문구 중복제거 전: 126
동대문구 중복제거 후: 126
동대문구 종료 : 2021-10-08 103053
동대문구 소요시간 : 0:00:42
------------------------------
중랑구 시작 : 2021-10-08 103054
중랑구 중복제거 전: 125
중랑구 중복제거 후: 125
중랑구 종료 : 2021-10-08 103131
중랑구 소요시간 : 0:00:37
------------------------------
성북구 시작 : 2021-10-08 103131
성북구 중복제거 전: 125
성북구 중